In [3]:
with open('../data/train_data/train.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
labels = []
sentences = []
sentence = ''
for line in lines:
    if line != '\n':
        labels.append(line[2:].strip('\n'))
        sentence += line[0]
    else:
        sentences.append(sentence)
        sentence = ''

100%|██████████| 2288790/2288790 [00:01<00:00, 1180096.38it/s]


In [4]:
with open('../data/preliminary_test_a/sample_per_line_train.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [1]:
with open('../data/train_data/train.txt', 'r', encoding='utf-8') as f:
    lines1 = f.readlines()
with open('./submit/labels_merged_res.txt', 'r', encoding='utf-8') as f:
    lines2 = f.readlines()
with open('./submit/extend_labels_merged_res_4w.txt', 'r', encoding='utf-8') as f:
    lines3 = f.readlines()
with open('./mix_train_data.txt', 'w', encoding='utf-8') as f:
    for line in lines1:
        f.write(line)
    for line in lines2:
        f.write(line)
    for line in lines3:
        f.write(line)

In [23]:
with open('../data/train_data/unlabeled_train_data.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
import random
random.shuffle(lines)
with open('./sample_per_line_extend_data.txt', 'w', encoding='utf-8') as f:
    for line in lines[:420000]:
        f.write(line)

### 训练集去噪